In [1]:
import psycopg2 as db

In [2]:
conn_string = "dbname='postgres' host='localhost' user='postgres' password='Tuan1211'"

In [3]:
conn = db.connect(conn_string)

In [47]:
conn.autocommit = True

In [48]:
cur = conn.cursor()

In [5]:
query = " insert into users (id, name, street, city, zip) values('{}','{}', '{}','{}', '{}')".format(1, 'Big Bird', 'Sesame Street', 'Fakeville', '12345')

In [6]:
print(cur.mogrify(query))

b" insert into users (id, name, street, city, zip) values('1','Big Bird', 'Sesame Street','Fakeville', '12345')"


In [49]:
#query2 = "insert into users (id,name,street,city,zip) values(%s,%s,%s,%s,%s)"
#data=(1,'Big Bird','Sesame Street','Fakeville','12345')
#print(cur.mogrify(query2,data))
#cur.execute(query2,data)

In [12]:
conn.commit()

In [14]:
from faker import Faker
fake = Faker()

In [15]:
import psycopg2 as db
data = []
i = 1

In [16]:
for r in range(1000):
    data.append((i, fake.name(), fake.street_address(), fake.city(), fake.zipcode()))
    i+=1

In [17]:
#convert ti tuples
data_for_db = tuple(data)

In [20]:
cur.executemany(query2,data_for_db)

In [21]:
conn.commit()

extract data

In [22]:
query = "select *from users"
cur.execute(query)

In [23]:
for i in cur:
    print(i)

(1, 'John Tran', 'East Danieltown', '465 Janice Pass Suite 475', '06099')
(2, 'April Carlson', 'South Emily', '088 Nicole Well Apt. 734', '99315')
(3, 'Joshua Conley DDS', 'New Frederickmouth', '98570 Ellis Mountains', '54324')
(4, 'Dr. David Johnson', 'North Loritown', '4147 Tina Summit', '73088')
(5, 'Kenneth Murray', 'Longburgh', '469 Garcia Inlet Apt. 292', '20342')
(6, 'Lisa Roberts', 'Alyssaburgh', '4038 Elliott Brooks Suite 522', '76734')
(7, 'Richard Smith', 'North Paul', '656 Jean Union', '07333')
(8, 'Curtis Rubio', 'Sarahside', '944 Gonzalez Street Apt. 773', '53408')
(9, 'Jason Gregory', 'New Thomas', '356 Tonya Loaf', '64875')
(10, 'Melissa Hayden', 'Cainborough', '9029 Shelly Parkway', '15736')
(11, 'Renee Williams', 'Williamston', '92994 Phillip Road Suite 251', '86005')
(12, 'Joseph Watson', 'North Kristenland', '911 Brandon Ford', '97696')
(13, 'Natalie Russell', 'Williamsmouth', '04549 Ruth Grove', '16027')
(14, 'Veronica Cole', 'Miketown', '203 Wise Flat', '27172')
(

In [24]:
cur.rowcount

1000

In [25]:
cur.rownumber

1000

In [32]:
sql = "COPY (SELECT *FROM users) TO STDOUT WITH CSV DELIMITER ','"

In [43]:
f = open("fromdb.csv", "w")

In [62]:
cur.copy_from(f, 'users', sep=',')

QueryCanceled: COPY from stdin failed: error in .read() call: UnsupportedOperation not readable
CONTEXT:  COPY users, line 1


In [51]:
import pandas as pd
df = pd.read_sql("select *from users", conn)

In [54]:
df

,id,name,city,street,zip
0,1,John Tran,East Danieltown,465 Janice Pass Suite 475,06099
1,2,April Carlson,South Emily,088 Nicole Well Apt. 734,99315
2,3,Joshua Conley DDS,New Frederickmouth,98570 Ellis Mountains,54324
3,4,Dr. David Johnson,North Loritown,4147 Tina Summit,73088
4,5,Kenneth Murray,Longburgh,469 Garcia Inlet Apt. 292,20342
...,...,...,...,...,...
995,996,Tristan Huffman,Jonesstad,896 William Drives Suite 283,20129
996,997,Bailey Flores,Martinberg,66162 Richardson View,52747
997,998,Karen Lester,Rachaelland,41429 Jennifer Inlet Apt. 864,42879
998,999,Kenneth Hill,New Nicholasport,222 Emily Shore Suite 174,10771


In [55]:
df.to_json(orient='records')

'[{"id":1,"name":"John Tran","city":"East Danieltown","street":"465 Janice Pass Suite 475","zip":"06099"},{"id":2,"name":"April Carlson","city":"South Emily","street":"088 Nicole Well Apt. 734","zip":"99315"},{"id":3,"name":"Joshua Conley DDS","city":"New Frederickmouth","street":"98570 Ellis Mountains","zip":"54324"},{"id":4,"name":"Dr. David Johnson","city":"North Loritown","street":"4147 Tina Summit","zip":"73088"},{"id":5,"name":"Kenneth Murray","city":"Longburgh","street":"469 Garcia Inlet Apt. 292","zip":"20342"},{"id":6,"name":"Lisa Roberts","city":"Alyssaburgh","street":"4038 Elliott Brooks Suite 522","zip":"76734"},{"id":7,"name":"Richard Smith","city":"North Paul","street":"656 Jean Union","zip":"07333"},{"id":8,"name":"Curtis Rubio","city":"Sarahside","street":"944 Gonzalez Street Apt. 773","zip":"53408"},{"id":9,"name":"Jason Gregory","city":"New Thomas","street":"356 Tonya Loaf","zip":"64875"},{"id":10,"name":"Melissa Hayden","city":"Cainborough","street":"9029 Shelly Park

In [58]:
from elasticsearch import Elasticsearch

In [59]:
es = Elasticsearch()

In [60]:
es=Elasticsearch({'127.0.0.1'})

In [61]:
doc={"name": fake.name(),"street": fake.street_address(), "city":
fake.city(),"zip":fake.zipcode()}
res=es.index(index="users",doc_type="doc",body=doc)
print(res['result']) `

created


/home/h2-server/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: the default number of shards will change from [5] to [1] in 7.0.0; if you wish to continue using the default of [5] shards, you must manage this on the create index request or with an index template
  warnings.warn(message, category=ElasticsearchWarning)


In [63]:
from elasticsearch import helpers

In [65]:
actions = [
    {
        "_index": "users",
        "_type": "doc",
        "_source": {
            "name": fake.name(),
            "street":fake.street_address(),
            "city": fake.city(),
            "zip": fake.zip()
        }
    }
    for x in range(998)
]

In [67]:
res = helpers.bulk(es, actions)

SerializationError: ({'name': 'Timothy Elliott', 'street': '506 Bell Mount', 'city': 'South Marie', 'zip': b'PK\x03\x04\x14\x00\x00\x00\x00\x00\xe8\x84\x96R\xe5\x8a\x11\x82\x00\x00\x01\x00\x00\x00\x01\x00\x15\x00\x00\x00TcMPkiDoVwdUlEyQaIkw1\t\xc3WO\xbe\xeb#vQ- \xd8P\x932#\xae9\x94\xd4\xb7n\x8b.|\xf3\x98\x01B\xa5\xc8@\x84\xb3\x87\x81\xa1fO\x17\xae\xbf\xeeP\xffh{\xe3[\xcdm\xf7\xbe\xa9\xb7\x0e\xa2[\x7f\xbc\x9e\xa3-X6\xedH\'6\x9aR\xc7>\xe96{\xc5\xd7\x00C\xcaC\xa5]\xec\x13\xf4]\xd5\x03a\x8b\xf3\xc1\xe3\xb7\x1eg\xfc\xe9\xbccd\xcdC\xb4\x0e\xaf\xf7+\xd5\x03c\x82;M\x99\xa2\xe1/d\xfa\xa2 (:W\x82\x89\xec\xa5\x0e\xd3c\x19\xd4\x9c\x8f\xa8q\x84:\x06\xaa\xa2\xb5x\xb3M\x7f\xd2`\x1bh?\xf7y\xde\x94:\xb2q\xb7|\xf9*\x11\xb2\xdc\xc3\xad\x8b\xa4jH\xea\x1c\xd6\x08\xe3\xa9%~&\xe7\x08hJm2\xa9a\xff\x96J\x1b[\x816"5J\x1c\xbd\xfb\xb3\xd3\x0f\x94@\x9b\x1a\xe8\xfcl\x94\xf6\xaf\x85\xd1\xcb\x11^r\x1a6OdIk\xd8\xe1\xa4@\xd4\xc7\xcd Jd\xed\xe4s\x94\xc5z\xf6l>\xf8\xc2!\xf8\x86\xd8\xf4`\x84q_\x93\x8b\xdf\xa5{\x1dn\xc7\xb